In [2]:
#!pip install psycopg2-binary
#!pip install pangres


In [3]:
from sqlalchemy import create_engine, text
import pandas as pd

# Parámetros de conexión
user = 'jbpm'
password = 'jbpm'
host = 'localhost'
port = '5432'
database = 'jbpm'

# conexión
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# Probar conexión y listar tablas
with engine.connect() as connection:
    result = connection.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public'"))
    for row in result:
        # Imprimir usando el índice de la tupla
        #print(row[0])  # row[0] es el valor de la primera columna (table_name)
        print(row)


('booleanexpression',)
('audittaskimpl',)
('bamtasksummary',)
('correlationkeyinfo',)
('casefiledatalog',)
('caseidinfo',)
('caseroleassignmentlog',)
('content',)
('contextmappinginfo',)
('deadline',)
('attachment',)
('delegation_delegates',)
('deploymentstore',)
('eventtypes',)
('errorinfo',)
('email_header',)
('executionerrorinfo',)
('i18ntext',)
('nodeinstancelog',)
('processinstancelog',)
('querydefinitionstore',)
('organizationalentity',)
('processinstanceinfo',)
('reassignment',)
('notification',)
('notification_bas',)
('notification_email_header',)
('notification_recipients',)
('peopleassignments_bas',)
('peopleassignments_exclowners',)
('peopleassignments_potowners',)
('peopleassignments_recipients',)
('peopleassignments_stakeholders',)
('reassignment_potentialowners',)
('sessioninfo',)
('requestinfo',)
('taskdef',)
('taskevent',)
('taskvariableimpl',)
('variableinstancelog',)
('workiteminfo',)
('task_comment',)
('task',)
('escalation',)
('correlationpropertyinfo',)
('tablajack

In [4]:
# #CARGO TODAS las Tablas y las encierro en un try except para que me arroje mensaje de error
# tablas = {
#     'tablajackone': None,
#     'tablajacktwo': None,
#     'tablajohn': None,
#     'tablakaty': None
# }

# try:
#     for nombre_tabla in tablas:
#         tablas[nombre_tabla] = pd.read_sql_table(nombre_tabla, con=engine)
#     print("✅ Success: todas las tablas cargadas correctamente.")
    
#  
#     dfjackone = tablas['tablajackone']
#     dfjacktwo = tablas['tablajacktwo']
#     dfjohn = tablas['tablajohn']
#     dfkaty = tablas['tablakaty']

# except Exception as e:
#     print(f"❌ Error: no se pudo cargar la tabla '{nombre_tabla}'. Detalle del error: {e}")


In [5]:
#Cagrar todas las talass
dfjackone = pd.read_sql_table('tablajackone', con=engine)
dfjacktwo = pd.read_sql_table('tablajacktwo', con=engine)
dfjohn = pd.read_sql_table('tablajohn', con=engine)
dfkaty = pd.read_sql_table('tablakaty', con=engine)
dfjackone.head()
# dfjacktwo.head()
# dfjohn.head()
# df.katy.head()

,id,lastname,name
0,1,Moreno Hernandez,Danika
1,2,prueba-TAreas,Prueba-tarea
2,3,tercerregistro,tercerregistro
3,4,registros duplicados,registros duplicados
4,5,NO DUPLICADOS,NO DUPLICADOS


In [6]:
from sqlalchemy import inspect
import pandas as pd

# Crear el inspector para leer la metadata de la base de datos
inspector = inspect(engine)

# Verificar si la tabla 'tablacontodos' existe
if 'tablacontodos' not in inspector.get_table_names():
    # Lista de tablas de origen
    tablas_origen = ['tablajackone', 'tablajacktwo', 'tablakaty', 'tablajohn']

    # Conjunto para guardar todos los nombres de columnas sin duplicados
    todas_las_columnas = set()

    # Iterar sobre cada tabla y obtener columnas
    for tabla in tablas_origen:
        columnas = [col['name'] for col in inspector.get_columns(tabla)]
        todas_las_columnas.update(columnas)

    # Lista de columnas deseadas
    columnas_deseadas = ['id', 'lastname', 'name', 'comments', 'email', 'date', 'reason', 'activo', 'inside']

    # Filtrar las columnas deseadas que están presentes en el conjunto
    columnas_finales = [col for col in columnas_deseadas if col in todas_las_columnas]

    # Crear un DataFrame vacío con las columnas deseadas
    df_vacio = pd.DataFrame(columns=columnas_finales)

    # Crear la tabla si no existe (vacía)
    df_vacio.to_sql('tablacontodos', engine, if_exists='fail', index=False)

    print(f"✅ La tabla 'tablacontodos' se ha creado con estas columnas:\n{columnas_finales}")
else:
    print("ℹ️ La tabla 'tablacontodos' ya existe. Continuando con el proceso...")


ℹ️ La tabla 'tablacontodos' ya existe. Continuando con el proceso...


In [7]:
dfcont = pd.read_sql_table('tablacontodos', con=engine)
dfcont.head()

,activo,comments,date,email,id,inside,lastname,name,reason


In [8]:
df_combined = dfjackone.merge(dfjacktwo, on='id', how='outer') \
                       .merge(dfkaty, on='id', how='outer') \
                       .merge(dfjohn, on='id', how='outer')
df_combined.head(0)

,id,lastname,name,comments,email,date,reason,activo,inside


In [9]:
#Eliminar duplicados
df_combined.drop_duplicates(subset='id', keep='last', inplace=True)

In [10]:
#Cargar el dataFrame en una tabla temporal porque la quiero comparar con "tablacontodos" y hacerle un join
def cargar_temporal(df_combined):
  
    df_combined.to_sql('temp_combined', engine, if_exists='replace', index=False)
    print("Datos cargados en la tabla temporal 'temp_combined'.")


In [11]:

# #df_concat = pd.concat([dfjackone, dfjacktwo,dfjohn,dfkaty], ignore_index=False)
# # Lista de DataFrames a combinar
# from functools import reduce

# dfs = [dfjackone, dfjacktwo, dfjohn, dfkaty]

# # Función para combinar dos DataFrames
# def merge_two(left, right):
#     return pd.merge(left, right, on='id', how='outer')

# # Combinación de todos los DataFrames
# df_concat = reduce(merge_two, dfs)

# #eliminar duplicados
# #df_concat = df_concat.drop_duplicates()
# df_concat = df_concat.drop_duplicates(subset=['id'])

# # Mostrar las primeras filas del DataFrame combinado
# df_concat.head()


In [12]:
#hago mi funcion que haga el left join a la tablacontodos para ver registros que no coinciden
def obtener_nuevos_registros():
    # Consulta SQL 
    query = """
    SELECT *
    FROM temp_combined AS t1
    LEFT JOIN tablacontodos AS t2 ON CAST(t1.id AS int) = CAST(t2.id AS int)
    WHERE t2.id IS NULL;
    """
    nuevos_registros = pd.read_sql(query, engine)
    return nuevos_registros


In [13]:
#INSERTAR REGISTROS QUE se encuentran en df_combined pero no estan en tablacontodos ya que son los nuevos
def insertar_nuevos_registros(nuevos_registros):
    if not nuevos_registros.empty:
        nuevos_registros.to_sql('tablacontodos', engine, if_exists='append', index=False)
        print(f"Se insertaron {len(nuevos_registros)} nuevos registros.")
    else:
        print("No hay registros nuevos para insertar.")


In [1]:
dfv = pd.read_sql("select processinstanceid , value, variableinstanceid  from variableinstancelog;", engine)
dfjackone.head()


NameError: name 'pd' is not defined

In [ ]:
#dfjacktwo = pd.read_sql("SELECT * FROM tablajacktwo", engine)
#dfjacktwo.head()


In [ ]:
#dfjohn = pd.read_sql("SELECT * FROM tablajohn", engine)
#dfjohn.head()


In [ ]:
#dfkaty= pd.read_sql("SELECT * FROM tablakaty", engine)
#dfkaty.head()


In [ ]:
#Ver si se encuentra en variables locales
#'dfjackone' in locals()


In [ ]:
# #Actualizar y crear la tablacontodos
# df_combined.to_sql(
#     name='tablacontodos',    # Nombre de la tabla en PostgreSQL
#     con=engine,              # Conexión a la base de datos
#     if_exists='append',      # Crea la tabla o la reemplaza si ya existe
#     index=False              # No incluir el índice de pandas como columna
# )


In [ ]:
#from sqlalchemy import text
#Verificar que la tabla ha sido insertada en la BD
with engine.connect() as connection:
    result = connection.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public'"))
    for row in result:
        print(row[0])


In [ ]:
# Reemplaza la tabla si ya existe (truncate + insert)
#df_combined.to_sql('tablacontodos', con=engine, if_exists='replace', index=False)



In [ ]:
#Ver la tablacontodos
df_verificacion = pd.read_sql('SELECT * FROM "public"."tablacontodos"', engine)
df_verificacion.head(50)
